In [53]:
import preprocessing
import pandas as pd
from sklearn.model_selection import train_test_split
import sqlite3
import seaborn as sns

In [ ]:
filePath="./data/financial.db"
# dataframe = preprocessing.loadAllData(filePath)

In [122]:
def extractXYValues(df, yVectorName):
    notNanDF = df[ df[yVectorName].notna() ]
    yValues = notNanDF[yVectorName]
    xValues = notNanDF.loc[:, notNanDF.columns != yVectorName] 

    one_hot_Xdata = pd.get_dummies(xValues)

    return (one_hot_Xdata, yValues)

def getTrainTestSplit(xValues, yValues): 
    X_train, X_test, y_train, y_test = train_test_split(xValues, yValues, test_size=0.33, random_state=42)

    return (X_train, X_test, y_train, y_test)


def accChangeToDemo(filepath):
    """
        Creates column to be added to demographic table. Column is taken from the 
        summed difference of the median account balances by district (1994-1998)
        relative to the median account value in all data in 1994.
    Args:
        filepath (str): path to financial.db.
    """
    conn = sqlite3.connect(filepath)
    accdist = pd.read_sql_query("SELECT account_id, district_id from account", conn)
    df = pd.read_sql_query("SELECT date as 'y-m-d', balance, account_id from trans ", conn)
    from datetime import date
    df['y-m-d'] = pd.to_datetime(df['y-m-d'])
    df['year'] = df['y-m-d'].map(lambda x: x.strftime('%Y'))
    df = pd.merge(accdist, df, on='account_id')
    
    # Gets the average value of each account per id per district per year
    avg = df.groupby(['year', 'district_id', 'account_id'])[['balance']].mean()

    # Calculates average account balance of each district per year
    average = avg.groupby(['year', 'district_id'])[['balance']].mean()
    
    # Gets difference of each account's median per year
    districtchange = average.groupby(['district_id'])[['balance']].diff().fillna(0)
#     asum = districtchange.groupby(['district_id'])[['balance']].sum()
#     newdemo = asum.div(302.70801242)
    percentChanged = (districtchange.loc["1998"] - districtchange.loc["1994"]) / districtchange.loc["1994"] * 100

    return percentChanged

In [123]:
percentChangedDF = accChangeToDemo(filePath)

In [110]:
conn = sqlite3.connect(filePath)
districtDF = pd.read_sql_query("SELECT * from district", conn)

In [111]:
conn.close()

In [124]:
percentChangedDF.rename(columns={'balance': 'percentChanged'}, inplace=True)

In [126]:
mergedDF = districtDF.merge(percentChangedDF, on=["district_id"])

In [127]:
mergedDF

,district_id,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,percentChanged
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107,-53.192741
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.6,1.85,132,2159.0,2674,-69.530230
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.9,2.21,111,2824.0,2813,-113.667532
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.6,5.05,109,5244.0,5892,-66.132741
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,9307,3.8,4.43,118,2616.0,3040,-95.782198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,73,Opava,north Moravia,182027,17,49,12,2,7,56.4,8746,3.3,3.74,90,4355.0,4433,-58.275734
73,74,Ostrava - mesto,north Moravia,323870,0,0,0,1,1,100.0,10673,4.7,5.44,100,18782.0,18347,-51.162989
74,75,Prerov,north Moravia,138032,67,30,4,2,5,64.6,8819,5.3,5.66,99,4063.0,4505,354.178303
75,76,Sumperk,north Moravia,127369,31,32,13,2,7,51.2,8369,4.7,5.88,107,3736.0,2807,-70.036111
